# Part 1: Producing the data
In this task, we will implement one Apache Kafka producer to simulate real-time data streaming. Spark is not allowed/required in this part since it’s simulating a streaming data source.

1. Your program should send one batch of applications every 5 seconds. One batch consists of a random 100-500 rows from the application_data_stream dataset. Note that only the number of rows needs to be random, you can read the file sequentially.  
    As an example, in the first and second batches, assuming we generate random numbers 100 and 400, the first batch will send records 1-100 from the CSV file, and the second batch will send records 101-500.  
    The CSV shouldn’t be loaded to memory at once to conserve memory (i.e. Read rows as needed).  
2. Add an integer column named ‘ts’ for each row, a Unix timestamp in seconds since the epoch (UTC timezone). Spead your batch out evenly for 5 seconds.  
    For example, if you send a batch of 100 records at 2024-02-01 00:00:00 (ISO format: YYYY-MM-DD HH:MM:SS) -> (ts = 1704027600):
    - Record 1-20: ts = 1704027600 
    - Record 21-40: ts = 1704027601 
    - Record 41-60: ts = 1704027602
    - ….
3. Send your batch to a Kafka topic with an appropriate name.

All the data except for the ‘ts’ column should be sent in String type, without changing to other data types. In many streaming processing applications, the data sources usually have little to no processing power (e.g. sensors). To simulate this, we shouldn’t do any processing/transformation at the producer.


In [ ]:
#import statements

from time import sleep
import random
import datetime as dt
import csv
from json import dumps
from kafka3 import KafkaProducer

#configuration
hostip = "kafka"
topic = 'application_data_stream'

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, value=data)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message:', str(ex))

def connect_kafka_producer():
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                 value_serializer=lambda x: dumps(x).encode('utf-8'))
        return producer
    except Exception as ex:
        print('Exception while connecting to Kafka:', str(ex))
        return None

def read_csv_file(file_name):
    rows = []
    with open(file_name, 'rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            rows.append(row)
    return rows

def generate_timestamp(base_timestamp, index, start_index):
    return base_timestamp + index - start_index
#inserting the csv file name application_data_stream.csv
if __name__ == '__main__':
    topic = 'application_data_stream'
    cRows = read_csv_file('application_data_stream.csv')
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    start_index = 0
#random the 
    while True:
        batch_size = random.randint(100, 500)
        base_timestamp = int(dt.datetime.now().timestamp())
        
        for i in range(start_index, start_index + batch_size):
            cRows[i]['ts'] = generate_timestamp(base_timestamp, i, start_index)
        
        data = cRows[start_index:start_index + batch_size]
        publish_message(producer, topic, data)

       #display the timestamps being sent
        for d in data:
            print(dt.datetime.fromtimestamp(d['ts']), ':', d)
        print('-------------------------')
   #reset to start from begining
        start_index += batch_size
        if start_index >= len(cRows):
            start_index = 0
        sleep(5)



In [ ]:
 # Close Kafka producer after publishing
producer.close()

# Reference

http://127.0.0.1:5202/notebooks/Lab/Lab%209/1.%20Kafka_Producer.ipynb - Kafka Producer Example - Lab 9

http://127.0.0.1:5202/notebooks/Lab/Lab%2010/3.%20Kafka_Climate_Producer.ipynb - Lab 10
http://127.0.0.1:5202/notebooks/Lab/Lab%2010/3.%20Kafka_TERRA_Producer.ipynb - Lab 10

http://127.0.0.1:5202/notebooks/Lab/Lab%2011/1.%20Kafka%20Producer%20-%20Late%20Events.ipynb -lab 11 for csv file